In [1]:
import io
import os
import cv2
import uuid
import json
import torch
import numpy as np
from fastapi import FastAPI, File, UploadFile
from mmcls.apis import init_model, inference_model
from PIL import Image
from mmcls.datasets.pipelines import Compose, Resize, LoadImageFromFile, RGBToGrayscale, CenterCrop, Normalize, ImageToTensor, Collect
from io import BytesIO
import base64



use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"
config_file = r'C:\Users\user\Documents\GitHub\mmclassification\configs\resnet\resnet50_8xb32_in1k copy.py'
checkpoint_file = r'C:\Users\user\Documents\GitHub\mmclassification\work_dir\resnet\latest.pth'
model = init_model(config_file, checkpoint_file, device=device)
model.CLASSES = ['Normal', 'Tuberculosis']


c:\Users\user\anaconda3\envs\mm\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


load checkpoint from local path: C:\Users\user\Documents\GitHub\mmclassification\work_dir\resnet\latest.pth


c:\users\user\documents\github\mmclassification\mmcls\apis\inference.py:45: UserWarning: Class names are not saved in the checkpoint's meta data, use imagenet by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [8]:

def preprocess_image(image_info):
	img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
	test_pipeline = [
		LoadImageFromFile(),
		Resize(size=(512,512)),
		CenterCrop(crop_size=450),
		RGBToGrayscale(keys=['img']),
		Normalize(**img_norm_cfg),
		ImageToTensor(keys=['img']),
		Collect(keys=['img'])
	]
	pipeline = Compose(test_pipeline)
	processed_data = pipeline(image_info)
	image = processed_data['img']
	return image[np.newaxis, :]

def predict_image(file: UploadFile = File(...)):
	# upload_path = r'./uploaded_img'

	# if not os.path.exists(upload_path):
	# 	os.makedirs(upload_path)
	
	# contents = file.read()
	# file_ext = os.path.splitext(file.filename)[-1]
	# unique_filename = f"{uuid.uuid4().hex}{file_ext}"
	# image_filepath = os.path.join(upload_path, unique_filename)

	# with open(image_filepath, "wb") as f:
	# 		f.write(contents)
	# image_info = dict(img_info=dict(filename=r'C:\Users\user\Documents\GitHub\mmclassification\TB_Chest_Radiography_Database\Tuberculosis\CHNCXR_0600_1.png'), img_prefix="")
	# image = preprocess_image(image_info)
	
	# image = np.array(image).reshape(3,image.shape[-1],image.shape[-1]).transpose(1,2,0)
	

	result = inference_model(model, r'C:\Users\user\Documents\GitHub\mmclassification\TB_Chest_Radiography_Database\Tuberculosis\CHNCXR_0600_1.png')
	result['pred_label'] = int(result['pred_label'])
	"""
	show_image = Image.open(image_filepath).convert('RGB')
	buffer = BytesIO()
	show_image.save(buffer, format=file_ext[1:])
	img_base64 = base64.b64encode(buffer.getvalue()).decode()
	result.update({
		'image' : img_base64
	})
	"""

	return {"result": result}



In [9]:
predict_image()

{'result': {'pred_label': 1,
  'pred_score': 0.9868778586387634,
  'pred_class': 'Tuberculosis'}}